# CLI08 - Stream Workers

## Overview

Macrometa GDN allows you to integrate streaming data and take appropriate actions. Most stream processing use cases involve collecting, analyzing, and integrating or acting on data generated during business activities by various sources.

| Stage | Description | 
|  :----:  |    :----:   |
| Collect | Receive or capture data from various data sources.| 
| Analyze | Analyze data to identify interesting patterns and extract information.| 
| Act | Take actions based on the findings. For example, running simple code, calling an external service, or triggering a complex integration.| 
| Integrate | Provide processed data for consumer consumption.| 

You can process streams to perform the following actions with your data:

- Transform data from one format to another. For example, from XML to JSON.
- Enrich data received from a specific source by combining it with databases and services.
- Correlate data by joining multiple streams to create an aggregate stream.
- Clean data by filtering it and by modifying the content in messages. For example, obfuscating sensitive information.
- Derive insights by identifying event patterns in data streams.
- Summarize data with time windows and incremental aggregations.
- Real-time ETL for collections, tailing files, and scraping HTTP endpoints.
- Integrating stream data and trigger actions based on the data. This can be a single service request or a complex enterprise integration flow.

In this tutorial we will build a simple stream worker for finding various heart rate measures like average bpm, minimum bpm etc.

## Pre-requisite

Lets Assume 
- you have already made a tenant account, and have a username and password
- you have installed the Macrometa CLI as explained in section 01
- you have generated an API Key as explained in section 01

In [ ]:
npm install -g gdnsl

## 1. Importing Libraries & Define Variables

The first step is to import the libraries we need and define the variables we will be using in this tutorial. This is also the right place to add your GDN login credentials. i.e. your email and password. You will also need to make sure you have specified the correct federation URL. In this example it is "gdn.paas.macrometa.io" and we are using the default geo fabric "_system".

In [86]:
# Variables - Queries
url="https://gdn.paas.macrometa.io";
apiKey="XXXX"; # apiKey goes here if applicable
email="email"; # Email goes here
# You can either use LOCAL or ALL also Please enter the name of the regions. For multiple regions enter comma-separated names. For example: region1, region2.
regions="ALL"
heart_rates_collection="HeartRates";
heart_rates_statistics_collection="HeartRateStatistics";
heart_rate_statistics_worker="HeartRateStatisticsWorker";
mock_heart_rate_data_generator="MockHeartRateDataGenerator";

## 2. Connecting to GDN

Now that we have imported the required libraries and added our login details, we can connect to GDN. do this by running the cell bellow.

You will see the cell output reflect a successful connection. If not go back to the first step and check the details you entered.

In [ ]:
echo "Creating gdnsl.yaml file" 

echo "url: $url
email: $email
apikey: $apiKey
regions:
  - $regions" > gdnsl.yaml

echo "------- CONNECTION SETUP  ------"
# if you are running this from terminal then you can ignore above step and run below command
# gdnsl init

## 3. Creating Collections

For this Tutorial we require the creating of two collections. The first will be a collection for the heart rates, which will hold our input data. The second will be the collection for the stream worker output which will hold the statistics.

Run the next cell to create these two collections. Note the If-Else statements that are included in this cell. This lets us check to see if the collections exist already in the federation you are using, if they dont exist we will create them.

In [ ]:
echo " ------- CREATE GEO-REPLICATED COLLECTION  ------"
gdnsl collection create $heart_rates_collection --type doc --stream 
echo "Created collection: $heart_rates_collection"
gdnsl collection create $heart_rates_statistics_collection --type doc 
echo "Created collection: $heart_rates_statistics_collection"

**Note: for this to work you need to enable Stream from the Macrometa Dashboard. Do this by opening up the dashboard, select collections, find the collection named "HeartRates" and select "enable Stream"**

![disabledstream](./../images/disabled_stream.png)

![enabledstream](./../images/enabled_stream.png)


## 4. Validate Stream Application

Now that we have created our collections we need to decide how we want to simulate the heart rate information. in this section we offer two approaches for simulating this data. 

Option 1, is to use a 3rd party tool called "Mockaroo" that will let you generate the data stream as an external source to Macrometa GDN. This would be closest to a "real world" use case showing data coming into GDN (and our streamworker) from an external source.

Option 2. demostrates how Macrometa GDN Stream Workers can also be used to provide simulation data, or indeed the streaming of data from a collection.

The end result will be the same, however for simplicity please select one option or the other, not both.

Select the option you are going to use, and run the code in the corresponding cell.

### 4.1 Validating heart rate simulator definition

#### Option1: Use mockaroo api to generate the mock heart rate data

NOTE: If you are using mockaroo API then you need to signup to mockaroo and paste the API Key. You can find API key here https://www.mockaroo.com/myaccount

In this Stream Worker we have created a trigger that triggers every 10 seconds and connects to the Mockaroo API Service and passes the current time stamp to trigger the Mockaroo data generation. 

Then the Stream Worker consumes data recived from the external Mockaroo service, selects the persons name and their heart rate and puts it into the collection "HeartRates".

Stream Workers are written in JavaScript, the Cell bellow has two variables one for yor Mackroo API Key, and the other "dataGeneratorAppDefinition", both are strings that are passed the Key and StreamWorker code, respectively.

In [ ]:
mockarooAPIKey="XXXX";

echo "--- Validate $mock_heart_rate_data_generator Application";

gdnsl stream-worker create --name $mock_heart_rate_data_generator \
--description "Mock data generator by calling mockaroo api for heart rate" \
--trigger "HeartRateDataGeneratorTrigger WITH ( interval = 10 sec );" \
--table "HeartRates (name string, bpm int);" \
--sink "MockarooServiceCallSink WITH (type='http-call', sink.id='mockaroo-service', publisher.url='https://api.mockaroo.com/api/a6e130b0?count=10&key=$mockarooAPIKey', map.type='json', method='GET') (triggered_time string);" \
--source "MockarooServiceResponseSink WITH (type='http-call-response', sink.id='mockaroo-service', map.type='json', http.status.code='200') (name string, bpm int);" \
--query "INSERT INTO MockarooServiceCallSink
SELECT time:currentTimestamp() as triggered_time 
FROM HeartRateDataGeneratorTrigger;" \
--query "INSERT INTO HeartRates SELECT name, bpm FROM MockarooServiceResponseSink;" \
--validate

#### Option2: Use custom stream worker to generate the heart rate data 

In this Stream Worker we have created a trigger that triggers every 10 seconds and creates random data from a short list of people and possible heart rates and puts it into the collection "HeartRates".

Stream Workers are written in JavaScript, the Cell bellow has one variables, "dataGeneratorAppDefinition", it is a string type variable that is passed StreamWorker code.

In [ ]:
echo "--- Validate $mock_heart_rate_data_generator Application";

gdnsl stream-worker create --name $mock_heart_rate_data_generator \
--description "Mock data generator by strem worker" \
--trigger "HeartRateDataGeneratorTrigger WITH ( interval = 10 sec );" \
--table "HeartRates (name string, bpm int);" \
--query "@info(name = 'ConsumeProcessedData')
    INSERT INTO HeartRates
    SELECT 
    js:eval(\"['Vasili', 'Rivalee', 'Betty', 'Jennifer', 'Alane', 'Sarena', 'Bruno', 'Carolee', 'Emmott', 'Andre'][Math.floor(Math.random() * 10)]\",\"string\") as name,
    js:eval(\"Math.floor(Math.random() * 40) + 40\",\"int\") as bpm
    FROM HeartRateDataGeneratorTrigger;" \
--validate

### 4.2 Validating stream worker

Now that we have selected and run the cell code for our chosen menthod of generating data, we can move on to the next stream worker, and then validate them for correctness, before we publish them, and make them active.

This next cell introduces the second stream worker that reads the heart rates from the collection "HeartRates" in a sliding window of 1 min, and then calculates the min max and averages for that window, and inserts them into the collection "HeartRateStatistics". it is defined as a string and assigned to the "statisticsAppDefinition" variable.

We then validate the streams by using the "validate_stream_app" method, as shown below.

In [ ]:
echo "--- Validate $heart_rate_statistics_worker Application";
  
# The stream app will be created by default in the local region. Optionally, you can send dclist to deploy stream
gdnsl stream-worker create --name $heart_rate_statistics_worker \
--description "Calculate the statistics for heart rates" \
--source "HeartRates WITH (type = 'database', collection = 'HeartRates', collection.type='doc', replication.type='global', map.type='json') (name string, bpm int);" \
--table "HeartRateStatistics (eventTime long, name string, minBpm int, maxBpm int, avgBpm double);" \
--query "INSERT INTO HeartRateStatistics
                SELECT 
                    eventTimestamp() as eventTime,
                    name as name,
                    min(bpm) as minBpm,
                    max(bpm) as maxBpm,
                    avg(bpm) as avgBpm
                FROM HeartRates window sliding_time(1 min)
                group by name" \
--validate
                

## 5. Save Stream Application

Upon a successful validation we can save our streamworkers by running the following cell.

### 5.1 Creating heart rate simulator definition

#### Option1: Use mockaroo api to generate the mock heart rate data 

In [ ]:
echo "--- Create $mock_heart_rate_data_generator Application";

gdnsl stream-worker create --name $mock_heart_rate_data_generator \
--description "Mock data generator by calling mockaroo api for heart rate" \
--trigger "HeartRateDataGeneratorTrigger WITH ( interval = 10 sec );" \
--table "HeartRates (name string, bpm int);" \
--sink "MockarooServiceCallSink WITH (type='http-call', sink.id='mockaroo-service', publisher.url='https://api.mockaroo.com/api/a6e130b0?count=10&key=$mockarooAPIKey', map.type='json', method='GET') (triggered_time string);" \
--source "MockarooServiceResponseSink WITH (type='http-call-response', sink.id='mockaroo-service', map.type='json', http.status.code='200') (name string, bpm int);" \
--query "INSERT INTO MockarooServiceCallSink
SELECT time:currentTimestamp() as triggered_time 
FROM HeartRateDataGeneratorTrigger;" \
--query "INSERT INTO HeartRates SELECT name, bpm FROM MockarooServiceResponseSink;"

#### Option2: Use custom stream worker to generate the heart rate data

In [ ]:
echo "--- Validate $mock_heart_rate_data_generator Application";

gdnsl stream-worker create --name $mock_heart_rate_data_generator \
--description "Mock data generator by strem worker" \
--trigger "HeartRateDataGeneratorTrigger WITH ( interval = 10 sec );" \
--table "HeartRates (name string, bpm int);" \
--query "@info(name = 'ConsumeProcessedData')
    INSERT INTO HeartRates
    SELECT 
    js:eval(\"['Vasili', 'Rivalee', 'Betty', 'Jennifer', 'Alane', 'Sarena', 'Bruno', 'Carolee', 'Emmott', 'Andre'][Math.floor(Math.random() * 10)]\",\"string\") as name,
    js:eval(\"Math.floor(Math.random() * 40) + 40\",\"int\") as bpm
    FROM HeartRateDataGeneratorTrigger;"

### 5.2 Creating heart rate statistics stream worker

In [ ]:
echo "--- Create $heart_rate_statistics_worker Application";
  
# The stream app will be created by default in the local region. Optionally, you can send dclist to deploy stream
gdnsl stream-worker create --name $heart_rate_statistics_worker \
--description "Calculate the statistics for heart rates" \
--source "HeartRates WITH (type = 'database', collection = 'HeartRates', collection.type='doc', replication.type='global', map.type='json') (name string, bpm int);" \
--table "HeartRateStatistics (eventTime long, name string, minBpm int, maxBpm int, avgBpm double);" \
--query "INSERT INTO HeartRateStatistics
                SELECT 
                    eventTimestamp() as eventTime,
                    name as name,
                    min(bpm) as minBpm,
                    max(bpm) as maxBpm,
                    avg(bpm) as avgBpm
                FROM HeartRates window sliding_time(1 min)
                group by name"

## 6. Publish Stream Application

A saved stream worker does not mean it is active. For the Stream Worker to be active it needs to be Published. Run the code in the next cell to publish the strem worker and make it active.

In [ ]:
echo "Activating Stream Workers"
gdnsl stream-worker $heart_rate_statistics_worker --enable
gdnsl stream-worker $mock_heart_rate_data_generator --enable
echo "Activated Stream Workers"

## 7. Checking HeartRates documents using C8QL

Now we have activated our stream workers the application is live. However, because we are using a 1 min window, we need to wait that time before we check for records being created.

A good way to do this rather than counting 60sec, is to load the GDN dashboard, select the "HeartRate" Collection, then "stream" and monitor the console. as soon as their are changes, you will see them reflected here.

![monitoring_heart_rate](./../images/monitoring-HR-stream.png)

Alternatively, wait 60 sec, and run the code in the cell below.

In [ ]:
gdnsl query "FOR doc IN $heart_rates_collection LIMIT 0, 5 RETURN doc"

## 8. Checking HeartRateStatistics documents using C8QL

Now that we have started to collect data in the heartrate collection, we want to see our analysis. run the cell below to see the output from our stream worker!

In [ ]:
gdnsl query "FOR doc IN $heart_rates_statistics_collection LIMIT 0, 5 RETURN doc"

## 9.  Delete StreamApp and Collections

Almost done. Lets clean up after ourselves and remove the tutorial from GDN.

In [ ]:
echo " DELETE_DATA"
gdnsl stream-worker delete $heart_rate_statistics_worker;
gdnsl stream-worker delete $mock_heart_rate_data_generator;
gdnsl collection delete $heart_rates_collection;
gdnsl collection delete $heart_rates_statistics_collection;
echo "StreamApp and Collection deleted"

## Section Completed!

Congratulations! you have completed this tutorial.